In [ ]:
import pandas as pd
import numpy as np
import random

from IPython.display import display, Markdown, clear_output
from ipywidgets import Layout
import ipywidgets as widgets

In [ ]:
dfj = pd.read_csv('../data/mauczka_label_consensus.csv')

In [ ]:
project = widgets.Text(
       value='',
       description='Project',
        layout=Layout( width='auto'))
revision = widgets.Text(
       value='',
       description='Revision hash',
        layout=Layout( width='auto'))
text = widgets.Textarea(
       value='',
       description='Commit message',
        layout=Layout( width='auto', height='250px'))
text2 = widgets.HTML(
       value='',
       description='Labels',
       layout=Layout( width='auto'))
label_internal_quality = widgets.Checkbox(
       description='Perfective?',)
label_external_quality = widgets.Checkbox(
       description='Corrective?',)
button = widgets.Button(description='Set label')

progress = widgets.HTML(value='', description='Progress')
link = widgets.HTML(value='', description='Revision')


disagreements = len(dfj[(dfj['label_internal_quality1'] != dfj['label_internal_quality2']) | (dfj['label_external_quality1'] != dfj['label_external_quality2'])])

def load_next():
    """Sample from unlabeled messages in the sample, change values on ui elements."""

    sample = dfj[(dfj['label_internal_quality1'] != dfj['label_internal_quality2']) | (dfj['label_external_quality1'] != dfj['label_external_quality2'])].sample(n=1)
    disagreements_left = len(dfj[~dfj['label_internal_quality'].isna()])
    
    project.value = sample['project'].values[0]
    revision.value = sample['revision_hash'].values[0]
    text.value = sample['message'].values[0]
    
    # display randomly labels
    label1 = 1
    label2 = 2
    rnd = random.randint(1, 2)
    if rnd == 2:
        label2 = 1
        label1 = 2
    text2.value = 'perfective: {}<br/>corrective: {}<br/>perfective: {}<br/>corrective: {}'.format(sample['label_internal_quality{}'.format(label1)].values[0],
                                                                                                   sample['label_external_quality{}'.format(label1)].values[0],
                                                                                                   sample['label_internal_quality{}'.format(label2)].values[0],
                                                                                                   sample['label_external_quality{}'.format(label2)].values[0])

    progress.value = '{}/{}'.format(disagreements_left, disagreements)
    owner = 'apache'
    repo = project.value
    if project.value == 'tapiji':
        owner = 'tapiji'
    if project.value == 'mylyn-reviews':
        owner = 'eclipse'
        repo = 'mylyn.reviews'
    link.value = '<a href="https://github.com/{}/{}/commit/{}" target="_blank">{}</a>'.format(owner, repo, revision.value, revision.value)

    label_internal_quality.value = False
    label_external_quality.value = False

def set_label(button):
    """Set label after click and then load next."""
    dfj.loc[dfj['revision_hash'] == revision.value, 'label_internal_quality'] = label_internal_quality.value
    dfj.loc[dfj['revision_hash'] == revision.value, 'label_external_quality'] = label_external_quality.value
    dfj.loc[dfj['revision_hash'] == revision.value, 'has_label'] = True
    load_next()

load_next()  # init
button.on_click(set_label)

label_box = widgets.HBox([label_internal_quality, label_external_quality])
box = widgets.VBox([progress, project, link, text, text2, label_box, button], layout={'display': 'flex', 'flex_flow': 'column', 'align_items': 'stretch'})

# show it
display(box)

In [ ]:
dfj.to_csv('../data/mauczka_label_consensus.csv', index=False)